# Disegno della ricerca e potere statistico {#sec-bayesian-regression-prediction}

**Prerequisiti**

- Leggere *Regression and Other Stories* [@gelman2020regression].
  - Prestare particolare attenzione al capitolo 16, "Design and sample size decisions", che offrono una guida dettagliata al tema del “statistical power,” which when studied naively yields unrealistic expectations of success and can lead to the design of ineffective, noisy studies.

**Concetti e Competenze Chiave**

- bla bla

**Preparazione del Notebook**

In [5]:
import os
import logging
import pandas as pd
import numpy as np
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
from IPython import get_ipython
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import cmdstanpy
cmdstanpy.utils.get_logger().setLevel(logging.ERROR)
from cmdstanpy import CmdStanModel
import pingouin as pg
from sklearn.linear_model import LinearRegression

In [90]:

seed: int = sum(map(ord, "bayesian_regression_prediction"))
rng: np.random.Generator = np.random.default_rng(seed=seed)
sns.set_theme(palette="colorblind")
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = "retina"

# Get the home directory
home_directory = os.path.expanduser("~")
# Construct the path to the Quarto project directory
project_directory = os.path.join(home_directory, "_repositories", "psicometria")

def standardize(series):
    """Standardize a pandas series with n degrees of freedom"""
    return (series - series.mean()) / series.std(ddof=0)

## Introduzione {.unnumbered}

bla bla

## Problema della potenza statistica

La potenza statistica è definita come la probabilità, prima che uno studio venga eseguito, che un determinato confronto raggiunga la "significatività statistica" a un livello predefinito (tipicamente un p-value inferiore a 0.05), dato un certo effetto reale presunto. Un'analisi della potenza viene eseguita ipotizzando prima una dimensione dell'effetto, poi facendo alcune assunzioni sulla variazione dei dati e sulla dimensione del campione dello studio, e infine utilizzando calcoli di probabilità per determinare la probabilità che il p-value sia inferiore alla soglia.

La visione convenzionale suggerisce di evitare studi con bassa potenza perché è improbabile che abbiano successo. Tuttavia, supponiamo che uno studio abbia bassa potenza ma possa essere eseguito a un costo molto basso rispetto ai potenziali benefici che potrebbero derivare da un successo nella ricerca. In questo caso, un ricercatore potrebbe concludere che vale ancora la pena condurre uno studio con bassa potenza, considerandolo una scommessa che vale la pena tentare. Pertanto, quando i costi sono bassi, i ricercatori sono spesso inclini a rischiare, nella convinzione che un risultato positivo potrebbe portare grandi benefici (per la carriera del ricercatore). Tuttavia, questo non è necessariamente una buona idea, come verrà discusso successivamente.

### La maledizione del vincitore negli studi a bassa potenza

Il problema con il ragionamento convenzionale è che, in uno studio a bassa potenza, il presunto "successo" della significatività statistica può essere in realtà una trappola. Concentrandosi su confronti che sono statisticamente significativi, la comunità accademica e i singoli ricercatori ottengono una visione sistematicamente distorta e eccessivamente ottimistica del mondo.

In termini semplici, quando il segnale è debole e il rumore è alto, i modelli statisticamente significativi nei dati sono probabilmente errati, nel senso che i risultati difficilmente si replicheranno.

In termini tecnici, i risultati statisticamente significativi sono soggetti a errori di tipo M e di tipo S (si veda il @sec-errors-s-m). Uno studio a bassa potenza non ha praticamente alcuna possibilità di fornire informazioni utili, e possiamo affermarlo anche prima che i dati siano raccolti. Pertanto, un rischio chiave per uno studio a bassa potenza non è tanto che abbia poche probabilità di successo, ma piuttosto che un apparente successo mascheri un fallimento più grande. La pubblicazione di risultati rumorosi può contribuire alla crisi di replicazione quando queste affermazioni fragili crollano sotto un'analisi più attenta o non si manifestano in tentativi di replicazione.

### Ipotesi di dimensione dell'effetto

Un'altra sfida è che qualsiasi analisi della potenza o calcolo della dimensione del campione è condizionato da una dimensione dell'effetto presunta, che è l'obiettivo dello studio e quindi non è mai conosciuta in anticipo. Esistono diversi modi per scegliere una dimensione dell'effetto per eseguire un'analisi di uno studio pianificato. Una strategia è provare una gamma di valori coerenti con la letteratura precedente sull'argomento. Un'altra è decidere quale grandezza dell'effetto sarebbe di interesse pratico.

@gelman2021regression raccomandano di non prendere decisioni di progettazione basate sulla stima di un singolo studio rumoroso. È meglio utilizzare un insieme di informazioni da studi precedenti per prendere decisioni informate sulla potenza statistica e la dimensione del campione. @gelman2021regression notano come i risultati pubblicati tendono ad essere sovrastime degli effetti reali per vari motivi, tra cui il fatto che gli interventi vengono spesso testati su persone e in scenari in cui saranno più efficaci, e i risultati "statisticamente significativi" sono più probabilmente riportati e pubblicati, portando a sovrastime.

## Stimare la dimensione del campione

Prima di raccogliere i dati, può essere utile stimare la precisione delle inferenze che ci si aspetta di ottenere con una determinata dimensione del campione, o stimare la dimensione del campione necessaria per ottenere una certa precisione.

In sintesi, per avere una potenza dell'80%, il vero valore del parametro deve essere 2.8 errori standard lontano dal punto di confronto. Tuttavia, questi calcoli sono validi solo quanto le loro assunzioni, e solitamente non si conosce il vero valore del parametro prima di eseguire lo studio.

### Determinazione della dimensione del campione per un errore standard specificato

Supponiamo di voler stimare un valore medio della popolazione $\theta$ a partire dai dati $y_1, \dots, y_n$, un campione casuale di dimensione $n$. Una stima veloce di $\theta$ è la media campionaria $\bar{y}$, che ha un errore standard pari a $\sigma/\sqrt{n}$, dove $\sigma$ è la deviazione standard di $y$ nella popolazione.

Se l'obiettivo è ottenere un errore standard specifico per $\bar{y}$, allora la dimensione del campione deve essere almeno:

$$
n = \left(\frac{\sigma}{\text{s.e.}}\right)^2,
$$

dove $\text{s.e.}$ è l'errore standard desiderato.

Per capire questo risultato, iniziamo ricordando la formula dell'errore standard della media campionaria:

$$\text{s.e.}(\bar{y}) = \frac{\sigma}{\sqrt{n}},$$

dove:

- $\text{s.e.}(\bar{y})$ è l'errore standard della media campionaria,
- $\sigma$ è la deviazione standard della popolazione,
- $n$ è la dimensione del campione.

Ora, supponiamo di voler ottenere un errore standard specifico, che chiameremo semplicemente $\text{s.e.}$. Possiamo quindi impostare l'equazione:

   $\text{s.e.} = \frac{\sigma}{\sqrt{n}}$

Per isolare $n$, prima moltiplichiamo entrambi i lati per $\sqrt{n}$:

   $\text{s.e.} \cdot \sqrt{n} = \sigma$

Ora, eleviamo al quadrato entrambi i lati:

   $(\text{s.e.} \cdot \sqrt{n})^2 = \sigma^2$

Semplifichiamo il lato sinistro:

   $\text{s.e.}^2 \cdot n = \sigma^2$

6Infine, dividiamo entrambi i lati per $\text{s.e.}^2$:

   $n = \frac{\sigma^2}{\text{s.e.}^2}$

Questa può essere riscritta come:

   $n = \left(\frac{\sigma}{\text{s.e.}}\right)^2$

Quindi, se vogliamo ottenere un errore standard specifico per $\bar{y}$, la dimensione del campione deve essere almeno pari a $\left(\frac{\sigma}{\text{s.e.}}\right)^2$, dove $\text{s.e.}$ è l'errore standard desiderato.

Questa formula ci permette di calcolare la dimensione minima del campione necessaria per ottenere un determinato livello di precisione (misurato dall'errore standard) nella stima della media della popolazione.

### Dimensione del campione per una potenza dell'80%

Se l'obiettivo è ottenere l'80% di potenza per distinguere $\theta$ da un valore specificato $\theta_0$, allora la dimensione del campione richiesta può essere calcolata con una formula conservativa:

$$
n = \left(\frac{2.8 \sigma}{\theta - \theta_0}\right)^2.
$$

Qui, il valore $2.8$ deriva dalla somma dei quantili della distribuzione normale per l'80% di potenza e il 95% di confidenza.

Per derivare la formula per la dimensione del campione necessaria a ottenere l'80% di potenza per distinguere $\theta$ da un valore specificato $\theta_0$, seguiamo i passaggi seguenti.

I termini che useremo sono i seguenti:

- **$\theta$**: Il valore medio della popolazione che stiamo cercando di stimare.
- **$\theta_0$**: Il valore specificato con cui vogliamo confrontare $\theta$.
- **$\sigma$**: La deviazione standard della popolazione.
- **$\text{s.e.}$**: L'errore standard della media campionaria $\bar{y}$, che è dato da $\sigma/\sqrt{n}$.

L'obiettivo è determinare la dimensione del campione $n$ necessaria per avere una potenza dell'80% nel distinguere $\theta$ da $\theta_0$, con un livello di significatività del 5%.

La differenza $\theta - \theta_0$ è l'effetto che stiamo cercando di rilevare. L'errore standard associato a questa differenza è dato da:

$$
\text{s.e.} = \frac{\sigma}{\sqrt{n}}.
$$

La precisione della stima della differenza $\theta - \theta_0$ dipende dalla dimensione del campione $n$.

Per una potenza dell'80%, vogliamo che la probabilità di rifiutare l'ipotesi nulla (quando in realtà $\theta \neq \theta_0$) sia almeno dell'80%. Ciò corrisponde a un'area sotto la curva normale standard di $0.8$.

Il livello di significatività del 5% corrisponde a un valore critico di $1.96$ (dalla distribuzione normale standard), che è il valore z corrispondente al 97,5% della distribuzione (poiché stiamo lavorando con un test a due code).

Per ottenere l'80% di potenza, il valore osservato deve trovarsi non solo al di fuori dell'intervallo di confidenza del 95%, ma anche a una certa distanza dalla media, tale da coprire il restante 80% della distribuzione della differenza $\theta - \theta_0$. Questo valore z corrispondente al restante 80% è $0.84$.

Quindi, il valore z totale da considerare è:

$$
z_{\text{totale}} = 1.96 + 0.84 = 2.8.
$$

La differenza $\theta - \theta_0$ deve essere almeno $2.8$ volte l'errore standard per garantire l'80% di potenza. Pertanto, possiamo scrivere:

$$
\theta - \theta_0 = 2.8 \times \frac{\sigma}{\sqrt{n}}.
$$

Risolviamo per $n$:

$$
\sqrt{n} = \frac{2.8 \sigma}{\theta - \theta_0}.
$$

Elevando al quadrato entrambi i lati:

$$
n = \left(\frac{2.8 \sigma}{\theta - \theta_0}\right)^2.
$$

Questa formula ci dice che, per avere l'80% di potenza nel rilevare una differenza $\theta - \theta_0$ con una deviazione standard $\sigma$, dobbiamo avere una dimensione del campione pari a 
$n = \left(\frac{2.8 \sigma}{\theta - \theta_0}\right)^2$. Il valore $2.8$ deriva dalla somma dei quantili della distribuzione normale standard: $1.96$ (per un intervallo di confidenza del 95%) e $0.84$ (per la potenza dell'80%). Questo valore assicura che l'intervallo di confidenza per $\theta$ non includa $\theta_0$ nel 80% dei casi, il che ci dà l'80% di potenza.

#### Correzione per la distribuzione t e incertezze nella deviazione standard

Nelle analisi di progettazione, utilizziamo la distribuzione normale per la regressione lineare quando la deviazione standard residua $\sigma$ è conosciuta. Tuttavia, per studi molto piccoli, i gradi di libertà sono bassi, e la deviazione standard residua non è stimata con precisione dai dati. In questi casi, le incertezze inferenziali seguono la distribuzione t di Student anziché la normale. Di conseguenza, il valore $2.8$ deve essere sostituito con un numero più grande per tenere conto di questa ulteriore fonte di incertezza.

Ad esempio, per uno studio che confronta due gruppi di 6 pazienti ciascuno, i gradi di libertà sono 10. In Python, la somma dei quantili della distribuzione t con 10 gradi di libertà per l'80% di potenza e il 95% di confidenza è $3.1$, quindi $2.8$ viene sostituito da $3.1$ nei calcoli per la potenza dell'80%.

In [1]:
import scipy.stats as stats

# Calcola i valori critici dalla distribuzione t di Student con 10 gradi di libertà
df = 10

# Calcola il quantile per l'80% di potenza (quindi 0.8) e il 95% di confidenza (quindi 0.975)
t_value_80 = stats.t.ppf(0.8, df)
t_value_95 = stats.t.ppf(0.975, df)

# Somma dei due quantili
t_total = t_value_80 + t_value_95
t_total

3.1071966805155227

Utilizzando Python, abbiamo calcolato i valori critici dalla distribuzione t di Student con 10 gradi di libertà. La somma dei quantili per l'80% di potenza e il 95% di confidenza è pari a $3.107$. Questo valore sostituisce il valore $2.8$ nei calcoli per la potenza dell'80% quando si utilizzano piccoli campioni con una distribuzione t invece della distribuzione normale standard.

## Confronto di medie

Consideriamo ora il caso del confronto tra due medie. L'errore standard della differenza tra due medie campionarie $\bar{y}_1 - \bar{y}_2$ è dato da:

$$
\text{s.e.} = \sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}.
$$

Se supponiamo che le dimensioni dei campioni nei due gruppi siano uguali ($n_1 = n_2 = n/2$), l'errore standard diventa semplicemente:

$$
\text{s.e.} = \sqrt{\frac{\sigma_1^2 + \sigma_2^2}{n/2}} = \sqrt{\frac{2\sigma^2}{n}} = \frac{\sqrt{2}\sigma}{\sqrt{n}}.
$$

Quindi, la dimensione del campione richiesta per ottenere un errore standard specifico sarà:

$$
n = \frac{2(\sigma_1^2 + \sigma_2^2)}{\text{s.e.}^2}.
$$

Se inoltre assumiamo che la variazione all'interno di ciascun gruppo sia la stessa ($\sigma_1 = \sigma_2 = \sigma$), allora:

$$
\text{s.e.} = \frac{2\sigma}{\sqrt{n}}
$$

e la dimensione del campione richiesta è:

$$
n = \left(\frac{2\sigma}{\text{s.e.}}\right)^2.
$$

### Dimensione del campione per una differenza $\Delta$ con l'80% di potenza

Per ottenere l'80% di potenza, la differenza $\Delta$ che vogliamo rilevare deve essere abbastanza grande rispetto all'errore standard della differenza tra le medie. Dato che abbiamo già calcolato che il valore $2.8$ è la somma dei quantili per l'80% di potenza e il 95% di confidenza, possiamo scrivere:

   $$
   \Delta = 2.8 \times \text{s.e.}.
   $$

Sostituendo l'espressione per $\text{s.e.}$, otteniamo:

   $$
   \Delta = 2.8 \times \sqrt{\frac{2(\sigma_1^2 + \sigma_2^2)}{n}}.
   $$

Ora, risolviamo per $n$:

   $$
   \sqrt{n} = \frac{2.8 \times \sqrt{2(\sigma_1^2 + \sigma_2^2)}}{\Delta}.
   $$

Elevando al quadrato entrambi i lati, otteniamo:

   $$
   n = 2\left(\frac{2.8 \sqrt{\sigma_1^2 + \sigma_2^2}}{\Delta}\right)^2 = 2\left(\frac{\sigma_1^2 + \sigma_2^2}{\left(\frac{\Delta}{2.8}\right)^2}\right).
   $$

Se assumiamo che la variazione all'interno dei due gruppi sia la stessa ($\sigma_1 = \sigma_2 = \sigma$), allora possiamo semplificare ulteriormente la formula. In questo caso, abbiamo:

   $$
   n = 2\left(\frac{\sigma^2 + \sigma^2}{\left(\frac{\Delta}{2.8}\right)^2}\right) = 2\left(\frac{2\sigma^2}{\left(\frac{\Delta}{2.8}\right)^2}\right).
   $$

Semplificando, otteniamo:

   $$
   n = \left(\frac{5.6 \sigma}{\Delta}\right)^2.
   $$

Qui, $5.6$ è semplicemente $2 \times 2.8$.

Questa formula ci dice che, per rilevare una differenza $\Delta$ tra due gruppi con una potenza dell'80%, la dimensione del campione richiesta dipende dalla deviazione standard $\sigma$ comune a entrambi i gruppi e dalla differenza $\Delta$ che vogliamo rilevare.

- *Se la deviazione standard $\sigma$ è alta*: Avremo bisogno di un campione più grande per rilevare una differenza $\Delta$.
- *Se la differenza $\Delta$ è piccola*: Anche in questo caso avremo bisogno di un campione più grande per garantire che la differenza sia rilevabile con l'80% di potenza.

::: {#exm-}

Se la differenza $\Delta$ che vogliamo rilevare è pari a metà della deviazione standard ($\Delta = 0.5\sigma$), allora la formula ci dirà quanti partecipanti sono necessari in ciascun gruppo per rilevare questa differenza con l'80% di potenza. Ad esempio, se $\Delta = 0.5\sigma$, la dimensione totale del campione sarà:

$$
n = \left(\frac{5.6 \sigma}{0.5\sigma}\right)^2 = (11.2)^2 = 125.44 \approx 126.
$$

Quindi, servirebbero 63 partecipanti per gruppo.

:::

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [58]:
%load_ext watermark
%watermark -n -u -v -iv -m  

Last updated: Sun Aug 11 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

pandas    : 2.2.2
cmdstanpy : 1.2.4
seaborn   : 0.13.2
matplotlib: 3.9.1
pingouin  : 0.5.4
numpy     : 1.26.4
arviz     : 0.18.0
logging   : 0.5.1.2

